In [7]:
import cv2
import os
import paddle as paddle
import paddle.fluid as fluid
import PIL.Image as Image
import random
import numpy as np
import time
from werkzeug.utils import secure_filename
from flask import Flask, request
from flask_cors import CORS
import uuid
import json

In [8]:
def load_model(model_id):
    model_net_path = 'models/infer_model/'+str(model_id)+'/'
    place = fluid.CPUPlace()
    exe = fluid.Executor(place)
    [infer_program, feeded_var_names, target_var] = fluid.io.load_inference_model(dirname=model_net_path, executor=exe)
    return exe, infer_program , feeded_var_names , target_var

def model_infer(image,*args):
    exe, infer_program , feeded_var_names , target_var = args[0]
    image = np.array(image).reshape(-1,1,28,28).astype('float32')
    image = (image) /255.0
    result = exe.run(program = infer_program,
                    feed = {feeded_var_names[0]:image},
                     fetch_list = target_var
                    )
    return result

In [9]:
def find_formula(path,pz_level=150,pz_height=25,extend = 6):
    img = cv2.imread(path)
    #img = np.array(Image.fromarray(img).resize((1920,928)))
    pz_level = int(pz_level * (img.shape[1]/1080))
    pz_height = int(pz_height * (img.shape[0]/1080))
    extend = min(10,max(int(extend * ((img.shape[0]*img.shape[1])/(1920*1080))),5))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    area_size = gray.shape[0]*gray.shape[1]*0.05
    ret, binary = cv2.threshold(gray, gray[0][0]-70, 255,cv2.THRESH_BINARY_INV)
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (pz_level, pz_height))
    dilation = cv2.dilate(binary, element1, iterations = 1)
    region = []
    result = []
    cout = 0
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for i in range(len(contours)):
        cnt = contours[i]
        area = cv2.contourArea(cnt)
        if area<area_size:
            continue
        rect = cv2.boundingRect(cnt)
        x = int(rect[0])
        y = int(rect[1])
        length = int(rect[2])
        height = int(rect[3])
        box = [y,y+height,x,x+length]
        region.append(box)
        
    for box in region:
        cout+=1
        tmp_image = img[box[0]:box[1],box[2]:box[3]]
        area = 0
        for i in contours:
            area += cv2.contourArea(i)
        if area<area_size:
            continue
        result.append(tmp_image)
    return result

In [10]:
def delet_repet(box_list):
    def takeSecond(region):
        return region[0]
    box_list.sort(key=takeSecond)
    return box_list
def infer_image(image_list,args):
    restr = [] 
    for image in image_list:
        image = np.array(image)
        greys_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image_expend_height = int(np.array(greys_image).shape[0] * 0.3)
        arg_color_number = np.sum(greys_image[0])/greys_image.shape[1]
        duce_color = max(-(120 - arg_color_number)/1.5,0)
        _,thre_image = cv2.threshold(greys_image, greys_image[0][0]-70,255,cv2.THRESH_BINARY_INV)
        #thre_image = Image.fromarray(thre_image).resize((int(50*(image.shape[1]/image.shape[0])),50))
        thre_image = np.array(thre_image)
        element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_expend_height*0.1), image_expend_height))
        dilate_image = cv2.dilate(thre_image, element1, iterations = 1)
        area_size = np.array(dilate_image).shape[0]*np.array(dilate_image).shape[1]/500
        contours, hierarchy = cv2.findContours(dilate_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        box_list = []
        arg_area = 0
        for contour in contours:
            x,y,w,h = cv2.boundingRect(contour)
            area = cv2.contourArea(contour)
            if area < area_size:
                if h<np.array(greys_image).shape[0]*0.5:
                    continue
            if h<w*1.5 and area < area_size*1.5:
                continue
            arg_area += area
            box_list.append([x,y,w,h,area])
        arg_area = arg_area / len(box_list)
        box_list = delet_repet(box_list)
        cout = 0
        shape = 28
        ceshi_image = []
        if len(box_list) == 0:
            continue
        for box in box_list:
            x,y,w,h,a = box
            w+=5
            h+=5
            cout+=1
            num_image = thre_image[y:y+h,x:x+w]
            if w>h:
                new_shape = int(shape*1.3)
                tmpH = int(128*(h/w))
                tmpW = 128
                newH = int(new_shape*(h/w))
                newW = new_shape
            else:
                new_shape = int(shape*1.3)
                tmpW = int(128*(w/h))
                tmpH = 128
                newH = new_shape
                newW = int(new_shape*(w/h))
            num_image = np.array(Image.fromarray(num_image).resize((tmpW,tmpH)))
            #临时图片用来测量用###########################################
            tmp_image1 = Image.fromarray(num_image).resize((newW,newH))
            tmp_image = Image.new('L',(shape,shape))
            tmp_image.paste(tmp_image1,(int((shape-newW)/2),int((shape-newH)/2)))
            tmp_image = np.array(tmp_image)
            tmp_contours, _ = cv2.findContours(tmp_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            tmp_area = 0
            for tmp_contour in tmp_contours:
                tarea = cv2.contourArea(tmp_contour)
                tmp_area+=tarea
            ############################################################
            yh_count = max(int((130 - tmp_area)/15),2)
            num_image = cv2.blur(num_image,(yh_count,yh_count))#羽化
            ret, num_image = cv2.threshold(num_image, 10, 255,cv2.THRESH_BINARY)
            #num_image = Image.fromarray(num_image).resize((20,20))
            num_image = Image.fromarray(num_image).resize((newW,newH))
            huabu = Image.new('L',(shape,shape))
            huabu.paste(num_image,(int((shape-newW)/2),int((shape-newH)/2)))
            cout+=1
            ceshi_image.append(np.array(huabu))
        result = model_infer(ceshi_image,args)
        re_str = ""
        math_map = ['0','1','2','3','4','5','6','7','8','9','^','+','-','*','/',')','(','=','Z']
        for i in result[0]:
            re = np.argmax(i)
            if re == 18:
                continue
            re_str+=math_map[re]
        restr.append(re_str)
    return restr

In [11]:
app = Flask(__name__)
# 允许跨越访问
CORS(app)
args = load_model(26)

In [12]:
@app.route('/infer', methods=['POST'])
def infer():
    f = request.files['img1']
    # 保存图片
    save_father_path = 'images'
    img_path = os.path.join(save_father_path, str(uuid.uuid1()) + '.' + secure_filename(f.filename).split('.')[-1])
    if not os.path.exists(save_father_path):
        os.makedirs(save_father_path)
    f.save(img_path)
    s = find_formula(img_path,200,25)
    r = infer_image(s,args)
    os.remove(img_path)
    r = json.dumps(r)
    r = '{"result": '+str(r)+'}'
    return r

In [ ]:
ip = '127.0.0.1'
if __name__ == '__main__':
    # 启动服务，并指定端口号
    app.run(host = str(ip),port=870)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
